In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms
import time
from collections import OrderedDict
import ssl
from download_mnist import load
ssl._create_default_https_context = ssl._create_unverified_context

C:\Users\myaku\miniconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\myaku\miniconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\myaku\miniconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size = (5,5))
        self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, kernel_size = (5,5))
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, kernel_size = (4,4))

        self.L1 = nn.Linear(256,120)
        self.L2 = nn.Linear(120,84)
        self.L3 = nn.Linear(84,10)
        
        self.pool = nn.MaxPool2d(kernel_size=(2,2),stride=2)
        
        self.act = nn.ReLU()
        self.softmax = nn.LogSoftmax(dim=-1)
        
    def forward(self,x):
        #Layer1
        x = self.conv1(x)
        x = self.act(x)
        x = self.pool(x)
        
        #Layer2
        x = self.conv2(x)
        x = self.act(x)
        x = self.pool(x)
        
        #Layer3
        x = self.conv3(x)
        x = self.act(x)
        
        #Layer4
        x = x.view(x.size()[0], -1)
        x = self.L2(x)
        x = self.act(x)
        
        #Layer5
        x = self.L3(x)
        x = self.softmax(x)
        
        return x

In [4]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    count = 0
    lossFunc = nn.NLLLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = lossFunc(output,target)
        loss.backward()
        optimizer.step()
    print('Train Epoch: {} \tLoss: {:.6f}'.format(
                epoch,  loss.item()))

In [5]:
def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
# print('==> Preparing data..')
# x_train, y_train, x_test, y_test = load()
import torchvision.datasets as datasets
trainSet = datasets.MNIST(root='data',train=True,transform=torchvision.transforms.ToTensor(),download=True)
testSet = datasets.MNIST(root='data',train=False,transform=torchvision.transforms.ToTensor(),download=True)

In [7]:
print("Optimizer: SGD \n")
time0 = time.time()
# Training settings
batch_size = 128
epochs = 5
lr = 0.05
use_cuda = torch.cuda.is_available()
torch.manual_seed(100)
device = torch.device("cuda" if use_cuda else "cpu")

train_loader = torch.utils.data.DataLoader(trainSet,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(testSet,batch_size=batch_size,shuffle=False)

model = LeNet().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)

time1 = time.time() 
print ('Training and Testing time for SGD: %s seconds ' % (time1-time0), "\n\n")

Optimizer: SGD 

Train Epoch: 1 	Loss: 0.073226

Test set: Average loss: 0.0844, Accuracy: 9728/10000 (97%)

Train Epoch: 2 	Loss: 0.036375

Test set: Average loss: 0.0550, Accuracy: 9819/10000 (98%)

Train Epoch: 3 	Loss: 0.074217

Test set: Average loss: 0.0453, Accuracy: 9848/10000 (98%)

Train Epoch: 4 	Loss: 0.049258

Test set: Average loss: 0.0451, Accuracy: 9855/10000 (99%)

Train Epoch: 5 	Loss: 0.036656

Test set: Average loss: 0.0488, Accuracy: 9836/10000 (98%)

Training and Testing time for SGD: 81.41930413246155 seconds  




In [9]:
print("Optimizer: Adagrad \n")
time0 = time.time()
# Training settings
batch_size = 128
epochs = 5
lr = 0.05
use_cuda = torch.cuda.is_available()
torch.manual_seed(100)
device = torch.device("cuda" if use_cuda else "cpu")

train_loader = torch.utils.data.DataLoader(trainSet,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(testSet,batch_size=batch_size,shuffle=False)

model = LeNet().to(device)
optimizer = optim.Adagrad(model.parameters(), lr=lr, weight_decay=5e-4)

for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)

time1 = time.time() 
print ('Training and Testing time for Adagrad: %s seconds ' % (time1-time0), "\n\n")

Optimizer: Adagrad 

Train Epoch: 1 	Loss: 0.084327

Test set: Average loss: 0.0932, Accuracy: 9712/10000 (97%)

Train Epoch: 2 	Loss: 0.024540

Test set: Average loss: 0.0603, Accuracy: 9793/10000 (98%)

Train Epoch: 3 	Loss: 0.084185

Test set: Average loss: 0.0557, Accuracy: 9825/10000 (98%)

Train Epoch: 4 	Loss: 0.049981

Test set: Average loss: 0.0589, Accuracy: 9795/10000 (98%)

Train Epoch: 5 	Loss: 0.017290

Test set: Average loss: 0.0434, Accuracy: 9858/10000 (99%)

Training and Testing time for Adagrad: 114.32223296165466 seconds  




In [10]:
print("Optimizer: RMSprop \n")
time0 = time.time()
# Training settings
batch_size = 128
epochs = 5
lr = 0.05
save_model = False
use_cuda = torch.cuda.is_available()
torch.manual_seed(100)
device = torch.device("cuda" if use_cuda else "cpu")

train_loader = torch.utils.data.DataLoader(trainSet,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(testSet,batch_size=batch_size,shuffle=False)

model = LeNet().to(device)
optimizer = optim.RMSprop(model.parameters(), lr=lr, eps=0.1)

for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)

time1 = time.time() 
print ('Training and Testing time for RMSprop: %s seconds ' % (time1-time0), "\n\n")

Optimizer: RMSprop 

Train Epoch: 1 	Loss: 0.075394

Test set: Average loss: 0.1315, Accuracy: 9589/10000 (96%)

Train Epoch: 2 	Loss: 0.044737

Test set: Average loss: 0.0649, Accuracy: 9791/10000 (98%)

Train Epoch: 3 	Loss: 0.062914

Test set: Average loss: 0.0593, Accuracy: 9805/10000 (98%)

Train Epoch: 4 	Loss: 0.060107

Test set: Average loss: 0.0605, Accuracy: 9807/10000 (98%)

Train Epoch: 5 	Loss: 0.028163

Test set: Average loss: 0.0457, Accuracy: 9855/10000 (99%)

Training and Testing time for RMSprop: 97.56442308425903 seconds  




In [12]:
print("Optimizer: Adam \n")
time0 = time.time()
# Training settings
batch_size = 128
epochs = 5
lr = 0.05
save_model = False
use_cuda = torch.cuda.is_available()
torch.manual_seed(100)
device = torch.device("cuda" if use_cuda else "cpu")

train_loader = torch.utils.data.DataLoader(trainSet,batch_size=batch_size,shuffle=True)
test_loader = torch.utils.data.DataLoader(testSet,batch_size=batch_size,shuffle=False)

model = LeNet().to(device)
optimizer = optim.Adam(model.parameters(),lr=lr)

for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)

time1 = time.time() 
print ('Training and Testing time for Adam: %s seconds ' % (time1-time0), "\n\n")

Optimizer: Adam 

Train Epoch: 1 	Loss: 0.589719

Test set: Average loss: 0.4916, Accuracy: 8482/10000 (85%)

Train Epoch: 2 	Loss: 0.449713

Test set: Average loss: 0.4635, Accuracy: 8547/10000 (85%)

Train Epoch: 3 	Loss: 0.402128

Test set: Average loss: 0.4421, Accuracy: 8550/10000 (86%)

Train Epoch: 4 	Loss: 0.692701

Test set: Average loss: 0.4638, Accuracy: 8551/10000 (86%)

Train Epoch: 5 	Loss: 0.597164

Test set: Average loss: 0.4484, Accuracy: 8526/10000 (85%)

Training and Testing time for Adam: 99.66939949989319 seconds  


